## flag1

In [96]:
P_known = open('algo-gzip.py', 'rb').read()
P_known = P_known.replace(b"\n", b"\r\n")
C_known = open('algo-gzip.f58A66B51.py', 'rb').read()

keystream = bytes(p ^ c for p, c in zip(P_known, C_known)) # 1079字节

C_flag = open('flag1-2-3.f58A66B51.txt', 'rb').read()
P_flag = bytes([a^b for a,b in zip(keystream, C_flag)])
print(P_flag)

b'This file contains all flags of misc-ransomware and some garbage data.\r\nM7cYYMoLfUg0OyGbSxLJ6n9AZt4U8CaCevNuJzpOTFQCptFuw4AY6m0k2rFkmoKwNdo63AsabM0zh7uEegR2XvKcEniQeJ4VPY0sHjekItUAAIYuAkdWpuEHoRZaZ9EdX4BwaGpvDiMoIy2S4yD1iuZYZN67S1Z6ErSmejIXFAf7V9lf0aHhFhemZHtE5flag{yOu_NeeD_somE_bAsIc_CRyPto_kNOwlEdgE_bEfORE_WRiTiNG_RANSoMWARE_gUHHI6jc6vTRXzg7J4UX}mAYp2MB2ByOvy7YesY4oKFLjfaTk8eZt7PwyqXh31DsdpIwvmtkn6bpUWLymn5DU6EzVG3GoigomgLKbmmfbEdd9Aym5nrWD3GgyFIt2d3KL5AtETib1n6lPOW34JIf3m2E6a3htdV5izMpMdTrgGU3XQDbRfr8AiWMG92An8ffVMNJyUkC2eFImwjwJ63A7CFUQb9eDcoJwVthgU2i9JxJceY5LAh3nc36QQGtMtDOOBsRfWWZVpk74HWdyJAna2fOe72k1I6LFrnND7gvBMWntBa6UyOPJoOMNRwHRBMoqoQNDAHt0ybBO1m3XXZi5i2CTTzfwxBxyJ2huSt9dOT10ur3B3YpAJp1Qi52VHADevnuSc6XgyeeiVL2d2kPlQY3vzgK40THAcmrcFH59FOSwbFBuLjSSDJRbOmJCCF0ZS8AcxFmk4xLasez4TgPA92tIZgckPuskGAuPIX7wDWi5thcKqsOn2Y1uyc2dyOV3bbXmjKpemZmnF70nNcvhGwCnWD9yMj9J9kzdLlBu9RbVkcMaR3tSYxpfyUOS49UW7mnmBHiOjbMRG2KGBc4ilMmYtVa8RuXnxukYfb8hKA9axnh6v23inX3mh0U9Ku7s80MGpcgWb9NB1RFEF8msYfBFDc

In [99]:
len(P_known), len(C_known), len(C_flag)

(1079, 1591, 1815)

## flag2

In [106]:
import struct
import os
import zipfile
from datetime import datetime

def parse_local_file_header(data, offset):
    """解析ZIP文件的Local File Header"""
    if len(data) - offset < 30:
        raise ValueError("数据长度不足，无法解析Local File Header")
    
    # 解析固定30字节的Header
    signature, version, flags, compression, mod_time, mod_date, crc32, comp_size, uncomp_size, name_len, extra_len = \
        struct.unpack('<LHHHHHLLLHH', data[offset:offset+30])
    
    # 检查签名
    if signature != 0x04034b50:
        raise ValueError(f"错误的Local File Header签名: 0x{signature:08x}")
    
    # 读取文件名
    name_start = offset + 30
    name_end = name_start + name_len
    filename = data[name_start:name_end].decode('utf-8', errors='replace')
    
    # 读取额外字段
    extra_start = name_end
    extra_end = extra_start + extra_len
    
    # 计算数据开始位置
    data_start = extra_end
    
    return {
        'offset': offset,
        'signature': f'0x{signature:08x}',
        'version_needed': (version >> 8) + (version & 0xFF) / 10,
        'flags': flags,
        'compression_method': compression,
        'compression_name': get_compression_name(compression),
        'mod_time': parse_dos_time(mod_time, mod_date),
        'crc32': f'0x{crc32:08x}',
        'compressed_size': comp_size,
        'uncompressed_size': uncomp_size,
        'filename_length': name_len,
        'extra_field_length': extra_len,
        'filename': filename,
        'data_start_offset': data_start,
        'header_length': 30 + name_len + extra_len
    }

def get_compression_name(method):
    """获取压缩方法名称"""
    methods = {
        0: 'No Compression',
        1: 'Shrunk',
        2: 'Reduced 1',
        3: 'Reduced 2', 
        4: 'Reduced 3',
        5: 'Reduced 4',
        6: 'Imploded',
        8: 'Deflated',
        9: 'Enhanced Deflated',
        10: 'PKWare DCL',
        12: 'BZIP2',
        14: 'LZMA',
        18: 'IBM TERSE',
        19: 'IBM LZ77',
        98: 'PPMd'
    }
    return methods.get(method, f'Unknown ({method})')

def parse_dos_time(time, date):
    """解析DOS格式的日期时间"""
    try:
        second = (time & 0x1F) * 2
        minute = (time >> 5) & 0x3F
        hour = (time >> 11) & 0x1F
        
        day = date & 0x1F
        month = (date >> 5) & 0x0F
        year = ((date >> 9) & 0x7F) + 1980
        
        return f'{year:04d}-{month:02d}-{day:02d} {hour:02d}:{minute:02d}:{second:02d}'
    except:
        return 'Invalid date'

def analyze_zip_headers(filedata, header1_offset, header2_offset):
    """分析ZIP文件的两个Local File Header"""
    try:
        data = filedata
        print(f"文件总大小: {len(data)} 字节")
        print("=" * 80)
        
        # 解析第一个Header
        print(f"\n第一个文件头 (偏移: 0x{header1_offset:04x}, {header1_offset} 字节):")
        print("-" * 50)
        header1 = parse_local_file_header(data, header1_offset)
        print_header_info(header1)
        
        # 解析第二个Header  
        print(f"\n第二个文件头 (偏移: 0x{header2_offset:04x}, {header2_offset} 字节):")
        print("-" * 50)
        header2 = parse_local_file_header(data, header2_offset)
        print_header_info(header2)
        
        # 显示两个文件之间的关系
        print("\n文件关系分析:")
        print("-" * 50)
        analyze_relationship(header1, header2, data)
        
        return header1, header2
        
    except FileNotFoundError:
        print(f"错误: 文件 {filename} 不存在")
    except Exception as e:
        print(f"解析错误: {e}")

def print_header_info(header):
    """打印Header信息"""
    print(f"  文件名: {header['filename']}")
    print(f"  文件头长度: {header['header_length']} 字节")
    print(f"  数据起始偏移: 0x{header['data_start_offset']:04x}")
    print(f"  压缩方法: {header['compression_name']}")
    print(f"  所需版本: {header['version_needed']}")
    print(f"  标志: 0x{header['flags']:04x}")
    print(f"  修改时间: {header['mod_time']}")
    print(f"  CRC32: {header['crc32']}")
    print(f"  压缩后大小: {header['compressed_size']} 字节")
    print(f"  未压缩大小: {header['uncompressed_size']} 字节")
    print(f"  文件名长度: {header['filename_length']} 字节")
    print(f"  额外字段长度: {header['extra_field_length']} 字节")

def analyze_relationship(header1, header2, data):
    """分析两个文件之间的关系"""
    # 计算第一个文件的数据结束位置
    file1_data_end = header1['data_start_offset'] + header1['compressed_size']
    
    print(f"第一个文件数据范围: 0x{header1['data_start_offset']:04x} - 0x{file1_data_end-1:04x}")
    print(f"第二个文件头位置: 0x{header2['offset']:04x}")
    
    gap = header2['offset'] - file1_data_end
    if gap == 0:
        print("文件排列: 紧密排列 (无间隙)")
    elif gap > 0:
        print(f"文件排列: 有 {gap} 字节间隙")
    else:
        print("文件排列: 重叠 (可能数据大小不准确)")
    
    # 检查第二个文件数据是否完整
    file2_expected_end = header2['data_start_offset'] + header2['compressed_size']
    actual_file_size = len(data)
    
    if file2_expected_end <= actual_file_size:
        print(f"第二个文件数据完整，结束于: 0x{file2_expected_end:04x}")
    else:
        missing = file2_expected_end - actual_file_size
        print(f"第二个文件数据不完整，缺少 {missing} 字节")

In [107]:
C_zip = open("flag-is-not-stored-in-this-file.f58A66B51.zip", 'rb').read()
P_zip = bytes([a^b for a,b in zip(keystream, C_zip)])

header1_offset = 0              # 第一个Local File Header的偏移量
header2_offset = 1002           # 第二个Local File Header的偏移量

# 分析Headers
headers = analyze_zip_headers(P_zip, header1_offset, header2_offset)

文件总大小: 1079 字节

第一个文件头 (偏移: 0x0000, 0 字节):
--------------------------------------------------
  文件名: no-flag-here
  文件头长度: 42 字节
  数据起始偏移: 0x002a
  压缩方法: No Compression
  所需版本: 2.0
  标志: 0x0000
  修改时间: 2025-10-17 11:45:14
  CRC32: 0x4484e69e
  压缩后大小: 960 字节
  未压缩大小: 960 字节
  文件名长度: 12 字节
  额外字段长度: 0 字节

第二个文件头 (偏移: 0x03ea, 1002 字节):
--------------------------------------------------
  文件名: also-not-here
  文件头长度: 43 字节
  数据起始偏移: 0x0415
  压缩方法: Deflated
  所需版本: 2.0
  标志: 0x0000
  修改时间: 2025-10-17 11:45:14
  CRC32: 0x73290859
  压缩后大小: 90 字节
  未压缩大小: 30 字节
  文件名长度: 13 字节
  额外字段长度: 0 字节

文件关系分析:
--------------------------------------------------
第一个文件数据范围: 0x002a - 0x03e9
第二个文件头位置: 0x03ea
文件排列: 紧密排列 (无间隙)
第二个文件数据不完整，缺少 56 字节


In [108]:
import struct
from datetime import datetime

def datetime_to_dos_time(dt_str):
    """将日期时间字符串转换为DOS格式"""
    try:
        # 解析时间字符串 "2025-10-17 11:45:14"
        dt = datetime.strptime(dt_str, "%Y-%m-%d %H:%M:%S")
        
        # 转换为DOS格式
        year = dt.year - 1980
        month = dt.month
        day = dt.day
        hour = dt.hour
        minute = dt.minute
        second = dt.second // 2  # DOS时间以2秒为单位
        
        dos_date = (year << 9) | (month << 5) | day
        dos_time = (hour << 11) | (minute << 5) | second
        
        return dos_time, dos_date
    except:
        # 如果解析失败，返回默认值
        return 0x4a41, 0x54a5  # 2025-10-17 11:45:14 的DOS表示

def reconstruct_zip_tail(header1, header2):
    """重建ZIP文件尾部"""
    
    # 计算Central Directory偏移 (第二个文件数据结束后)
    cd_offset = header2['data_start_offset'] + header2['compressed_size']
    
    # 构建Central Directory
    central_directory = b''
    
    # 文件1的CD记录 - 分步打包避免错误
    time1, date1 = datetime_to_dos_time(header1['mod_time'])
    
    # 第一部分: 签名和版本信息 (8字节)
    cd1_part1 = struct.pack('<LHH',
        0x02014b50,           # signature
        20, 20                # version made by, version needed
    )
    
    # 第二部分: 标志和压缩方法 (4字节)
    cd1_part2 = struct.pack('<HH',
        0, 0                  # flags, compression method
    )
    
    # 第三部分: 时间信息 (4字节)
    cd1_part3 = struct.pack('<HH',
        time1, date1          # mod time/date
    )
    
    # 第四部分: CRC和大小信息 (12字节)
    cd1_part4 = struct.pack('<LLL',
        int(header1['crc32'], 16),  # crc32
        header1['compressed_size'] & 0xFFFFFFFF,
        header1['uncompressed_size'] & 0xFFFFFFFF
    )
    
    # 第五部分: 长度信息 (10字节)
    cd1_part5 = struct.pack('<HHHHH',
        len(header1['filename']),  # file name length
        0,                        # extra field length  
        0,                        # file comment length
        0,                        # disk number start
        0                         # internal file attributes
    )
    
    # 第六部分: 属性和偏移 (8字节)
    cd1_part6 = struct.pack('<LL',
        0,                       # external file attributes
        header1['offset'] & 0xFFFFFFFF  # local header offset
    )
    
    # 组合所有部分
    cd1 = cd1_part1 + cd1_part2 + cd1_part3 + cd1_part4 + cd1_part5 + cd1_part6
    cd1 += header1['filename'].encode('utf-8')
    central_directory += cd1
    
    # 文件2的CD记录 - 同样的分步打包
    time2, date2 = datetime_to_dos_time(header2['mod_time'])
    
    # 第一部分: 签名和版本信息
    cd2_part1 = struct.pack('<LHH',
        0x02014b50,
        20, 20
    )
    
    # 第二部分: 标志和压缩方法
    cd2_part2 = struct.pack('<HH',
        0, 8                    # compression method = deflated
    )
    
    # 第三部分: 时间信息
    cd2_part3 = struct.pack('<HH',
        time2, date2
    )
    
    # 第四部分: CRC和大小信息
    cd2_part4 = struct.pack('<LLL',
        int(header2['crc32'], 16),
        header2['compressed_size'] & 0xFFFFFFFF,
        header2['uncompressed_size'] & 0xFFFFFFFF
    )
    
    # 第五部分: 长度信息
    cd2_part5 = struct.pack('<HHHHH',
        len(header2['filename']),
        0, 0, 0, 0
    )
    
    # 第六部分: 属性和偏移
    cd2_part6 = struct.pack('<LL',
        0,
        header2['offset'] & 0xFFFFFFFF
    )
    
    # 组合所有部分
    cd2 = cd2_part1 + cd2_part2 + cd2_part3 + cd2_part4 + cd2_part5 + cd2_part6
    cd2 += header2['filename'].encode('utf-8')
    central_directory += cd2
    
    # 构建EOCD
    cd_size = len(central_directory)
    eocd = struct.pack('<LHHHHLLH',
        0x06054b50,           # signature
        0, 0,                 # disk numbers
        2, 2,                 # entries in disk, total entries
        cd_size,              # CD size
        cd_offset,            # CD offset
        0                     # comment length
    )
    
    return central_directory, eocd, cd_offset

In [110]:
cd, eocd, offset = reconstruct_zip_tail(headers[0], headers[1])
P_zip_tail = cd + eocd
tail_key = [a^b for a, b in zip(P_zip_tail, C_zip[1135: 1274])]
bytes([a^b for a,b in zip(tail_key, C_flag[1135: 1274])])

b'fvYY4wCFGSVIH4d05Xr2XZAcAjHo0vLqAjDWDNfl\xe1f{cORruPTeD_zIp_cAn_be_reCOvErEd_BuT_ReDUNdaNcY_alSo_LeaD\xd3^To_AmBiGuIty_OxShnyRcDUp1ogzv0aK2Q}LQyn'